In [1]:
#import packages
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.preprocessing as prep
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#load dataset
data_train = pd.read_csv('RBSHack2018-master/hackdata/hack_data_v1/train_indessa.csv')
data_train.shape

(532428, 45)

In [3]:
#Columns in data
#data_train.columns

In [4]:
X= data_train.iloc[:,0:44]
y= data_train.iloc[:,44]

In [5]:
#X.info()
#X.head()

### Data Preprocessing


In [6]:
#Drop Columns
drop_columns= ['member_id','batch_enrolled','sub_grade','emp_title','title','desc','zip_code']
X= X.drop(drop_columns,axis=1)
#X.shape

In [7]:
#Examine the missing values
def find_missing_values(data):
    missing_val = X.isnull().sum()
    missing_per = (missing_val/len(data))*100
    missing_tbl = pd.concat([missing_val, missing_per], axis=1)
    print(missing_tbl)

#find_missing_values(X)

In [8]:
def pre_process_data(X):
    
    encoded= pd.DataFrame()

    #Convert to numeric
    X['last_week_pay']= X['last_week_pay'].str.replace('th week','').replace('NA','0').astype(int)

    #Encode emp_length
    emp_length_mapping = {np.nan:0,'< 1 year':1,'1 year':2,'2 years':3,'3 years':4,'4 years':5,'5 years':6,'6 years':7,'7 years':8,'8 years':9,'9 years':10,'10+ years':11}
    encoded['emp_length_encoded'] = X['emp_length'].replace(emp_length_mapping)

    #Impute for Nominal variables
    nominal_columns= X[['term','grade','home_ownership','verification_status','pymnt_plan','purpose','addr_state','initial_list_status','application_type','verification_status_joint']]
    nominal_col= pd.get_dummies(nominal_columns)
    encoded = pd.concat([encoded,nominal_col],axis=1)

    #prep_train_data = pre_process_data(X)
    #Impute for Ordinal Variables
    ordinal_columns= X[['loan_amnt','funded_amnt','funded_amnt_inv','int_rate','annual_inc','dti','delinq_2yrs','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec','revol_bal','revol_util','total_acc','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','collections_12_mths_ex_med','mths_since_last_major_derog','acc_now_delinq','tot_coll_amt','tot_cur_bal','total_rev_hi_lim','last_week_pay']]
    ordinal_columns= ordinal_columns.apply(prep.LabelEncoder().fit_transform)
    encoded = pd.concat([encoded,ordinal_columns],axis=1)

    #Impute NULL value for float
    float_null_col = X[['annual_inc','delinq_2yrs','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec','revol_util','total_acc','collections_12_mths_ex_med','mths_since_last_major_derog','acc_now_delinq','tot_coll_amt','tot_cur_bal','total_rev_hi_lim']]
    mean_imputer = prep.Imputer(missing_values=np.nan,strategy='mean',axis=0)
    imputed_df = pd.DataFrame(mean_imputer.fit_transform(float_null_col))
    imputed_df.columns= float_null_col.columns
    imputed_df.index= float_null_col.index
    encoded= pd.concat([encoded,imputed_df],axis=1)

    return encoded

In [9]:
encoded = pre_process_data(X)
encoded = encoded.drop(['home_ownership_ANY'],axis=1)
#encoded.head()

# Modeling

In [11]:
#Import RandomForest
from sklearn.ensemble import RandomForestRegressor
#Import Error metrics
from sklearn.metrics import roc_auc_score

In [12]:
model = RandomForestRegressor(n_estimators=10, oob_score=True, n_jobs=-1, random_state=42)
model.fit(encoded,y)

MemoryError: 

model.oob_score_

In [ ]:
plt.figure(figsize=(5,5))
sns.heatmap(data=X.corr())
plt.show()
plt.gcf().clear()